In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [ ]:
def read_dat_file(file_name):
  cols_names = []
  # Read column names
  with open(file_name,'r') as f:
      lines = f.readlines()
      for line in lines:
        if "#" not in line:
          # From now onwards it will be all data
          break
        elif "=" in line:
          if ("[" not in line) and ("]" not in line):
             continue
          cols_names.append(line.split('=')[-1][1:-1].strip())
        else:
          continue

  return pd.read_csv(file_name,sep="\s+",comment="#",names=cols_names)

def find_topologies(path:Path):
  topologies_set = set()
  for i in path.iterdir():
    if i.is_file():
      topologies_set.add(i.stem.split("_")[0])

  return list(topologies_set)

def find_dat_file_names(path:Path):
  file_name_set = set()
  for i in path.iterdir():
    if i.is_file():
      file_name_set.add(i.stem.split("_")[1])

  return list(file_name_set)

def get_top_name_and_mode(name):
  # Bf0I1(12 modes).dat -> Bf0I1, 12
  top_name = name.split("(")[0]
  mode = int(name.split("(")[-1].split(" ")[0])
  return top_name,mode

def find_highest_modes_for_topologies(path:Path):
  highest_mode_dict = {}
  for i in path.iterdir():
    if i.is_file():
      top_name, mode = get_top_name_and_mode(i.stem)
      if top_name in highest_mode_dict:
        if highest_mode_dict[top_name] < mode:
          highest_mode_dict[top_name] = mode
      else:
        highest_mode_dict[top_name] = mode

  return highest_mode_dict

def make_mode_dataframe(path:Path):
  highest_mode_dict = find_highest_modes_for_topologies(path)
  top_dataframe_list = {i:[] for i in highest_mode_dict}

  for i in path.iterdir():
    for top_name in highest_mode_dict:
      if (top_name+"(") in i.stem:
        top_dataframe_list[top_name].append(read_dat_file(i))

  top_mode_df_dict = {}
  for i,df_list in top_dataframe_list.items():
    result = pd.concat(df_list, ignore_index=True)

    # Remove duplicates based on 't' column (keep first occurrence)
    # result = result.drop_duplicates(subset='t', keep='first')

    # Sort by 't' and reset index
    top_mode_df_dict[i] = result.sort_values('t').reset_index(drop=True)
  return top_mode_df_dict

# make_mode_dataframe(Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics/FilledCylinderCB0.dir/Powerpsi.dir"))

In [ ]:
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/6_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Powerpsi.dir/Bf0I1(19 modes).dat")
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Powerpsi.dir/Bf0I1(20 modes).dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Bf0I1_HighestThirdConvergenceFactor.dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/6_set1_L3_Lev3/extracted-PowerDiagnostics/SphereC0.dir/Bf0I1_TruncationError.dat")
# file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics/SphereC10.dir/Powerpsi.dir/Bf1S2(15 modes).dat")
file = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/16_set1_L3_HP32_AF_Lev3/extracted-PowerDiagnostics/SphereC1.dir/Powerkappa.dir/Bf0I1(15 modes).dat")

In [ ]:
# data = read_dat_file(file)
h5_path = Path('/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4')
# h5_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/h5_files_Lev3')
h5_path = Path('/groups/sxs/hchaudha/spec_runs/16_set1_L3/h5_files_Lev3/')
# h5_path = Path('/groups/sxs/hchaudha/spec_runs/16_set1_L3_HP28/h5_files_Lev3')
# h5_path = Path('/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L6/h5_files_Lev6/')

domain = 'FilledCylinderCB0'
domain = 'SphereC1'
# domain = 'SphereA0'

psi_or_kappa = 'kappa'
psi_or_kappa = 'psi'

folder_path = Path(f"{h5_path}/extracted-PowerDiagnostics/{domain}.dir/Power{psi_or_kappa}.dir")
top_data = make_mode_dataframe(folder_path)
print(top_data.keys())

In [ ]:
top_name = 'Bf1S2'
top_name = list(top_data.keys())[0]

data = top_data[top_name]
column_names = data.columns[1:]
visual_data = data[column_names]

plt.figure(figsize=(15, 10))
imshow_plot = plt.imshow(
    # visual_data, 
    np.log10(visual_data), 
    aspect='auto', 
    cmap='RdYlGn_r', 
    origin='lower',interpolation='none',
)

plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = 1
ytick_step = len(visual_data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(visual_data), ytick_step), 
    labels=data['t'][::ytick_step].astype(int)
)
plt.colorbar(imshow_plot)
plt.ylabel('t(M)')
plt.title(f'{str(folder_path)[31:]}_{top_name}')
plt.tight_layout() 
plt.grid(False)
plt.show()

In [ ]:
cols_to_use = [i for i in data.columns if 't' not in i]
df = np.log10(data[cols_to_use])
df['row_min'] = df.min(axis=1)
df['row_max'] = df.max(axis=1)
df['row_mean'] = df.mean(axis=1)
df['row_std'] = df.std(axis=1)

# plt.plot(df['row_min'])
# plt.plot(df['row_mean'])
# plt.plot(df['row_max'])
# plt.plot(df['row_std'])
plt.figure(figsize=(12, 8))

for i in cols_to_use:
  plt.plot(data['t'], df[f'{i}'])
  # plt.plot(data['t'], df[f'{i}'], label = f'{i}')
plt.legend()
plt.tight_layout()
plt.title(f'{str(folder_path)[31:]}_{top_name}')
plt.tight_layout() 
plt.grid(False)
plt.show()
# x = data['t']
# y = df['row_mean']
# y_err = df['row_std']
# plt.errorbar(x, y, yerr=y_err, fmt='-o', label='Data with Error Bars', ecolor='red', capsize=4)

## Reading the whole folder

In [ ]:
power_diag_path = Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics")

In [ ]:
find_highest_modes_for_topologies(Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics/FilledCylinderCB0.dir/Powerpsi.dir"))

In [ ]:
find_dat_file_names(Path("/groups/sxs/hchaudha/spec_runs/del/filtering/13_set1_L4_1500_Lev4/extracted-PowerDiagnostics/FilledCylinderCB0.dir"))